In [ ]:
import spacy
from spacy.tokens import DocBin
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:

# Load spaCy's language model if necessary (for example, the English model)
nlp = spacy.blank("en")  # Replace "en" with the appropriate language code

# Load your .spacy file
train_doc_bin = DocBin().from_disk("Data/train.spacy")
dev_doc_bin = DocBin().from_disk("Data/dev.spacy")
# Deserialize the docs
train_docs = list(train_doc_bin.get_docs(nlp.vocab))
dev_docs = list(dev_doc_bin.get_docs(nlp.vocab))

In [ ]:
# Now you can work with the docs
for doc in train_docs:
    for ent in doc.ents:
        print(ent.text, ent.label_)

In [ ]:
def getLabelsCounts(docs):
    labels = []
    for doc in docs:
        for ent in doc.ents:
            labels.append(ent.label_)

    # Convert the list of labels to a NumPy array
    labels_array = np.array(labels)
    unique_labels, counts = np.unique(labels_array, return_counts=True)
    counts = dict(zip(unique_labels, counts))
    return counts

In [ ]:
trainLabelsCounts = getLabelsCounts(train_docs)
devLabelsCounts = getLabelsCounts(dev_docs)

In [ ]:
def saveLabelsPie(LabelsCounts, name):
    plt.figure(figsize=(8, 8))
    colors = plt.cm.hsv(np.linspace(0, 1, len(LabelsCounts)))
    patches, texts, autotexts = plt.pie(LabelsCounts.values(), labels=LabelsCounts.keys(), 
            autopct='%1.1f%%', 
            colors=colors, 
            startangle=60,
            wedgeprops=dict(edgecolor='w'))
    for text in texts + autotexts:
        text.set_fontsize(9)
    plt.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
    plt.title(name, pad=30, fontdict = {'fontsize':20, 'fontstyle' : 'oblique'})
    plt.savefig(f"./Plots/{name}.png", bbox_inches='tight', transparent=True)
    plt.show()



In [ ]:
saveLabelsPie(trainLabelsCounts, "Named entity proportions in training")
saveLabelsPie(devLabelsCounts, "Named entity proportions in development")